In [ ]:
import cv2
import numpy as np
import imutils
import glob
import matplotlib.pyplot as plt
import os

THIS FUNCTION DOESNT WORK

In [ ]:
import numpy as np 
import cv2

# Check for left and right camera IDs
# These values can change depending on the system
CamL_id = 2 # Camera ID for left camera
CamR_id = 0 # Camera ID for right camera

CamL= cv2.VideoCapture(CamL_id)
CamR= cv2.VideoCapture(CamR_id)

# Reading the mapping values for stereo image rectification
cv_file = cv2.FileStorage("data/stereo_rectify_maps.xml", cv2.FILE_STORAGE_READ)
Left_Stereo_Map_x = cv_file.getNode("Left_Stereo_Map_x").mat()
Left_Stereo_Map_y = cv_file.getNode("Left_Stereo_Map_y").mat()
Right_Stereo_Map_x = cv_file.getNode("Right_Stereo_Map_x").mat()
Right_Stereo_Map_y = cv_file.getNode("Right_Stereo_Map_y").mat()
cv_file.release()

def nothing(x):
    pass

cv2.namedWindow('disp',cv2.WINDOW_NORMAL)
cv2.resizeWindow('disp',600,600)

cv2.createTrackbar('numDisparities','disp',1,17,nothing)
cv2.createTrackbar('blockSize','disp',5,50,nothing)
cv2.createTrackbar('preFilterType','disp',1,1,nothing)
cv2.createTrackbar('preFilterSize','disp',2,25,nothing)
cv2.createTrackbar('preFilterCap','disp',5,62,nothing)
cv2.createTrackbar('textureThreshold','disp',10,100,nothing)
cv2.createTrackbar('uniquenessRatio','disp',15,100,nothing)
cv2.createTrackbar('speckleRange','disp',0,100,nothing)
cv2.createTrackbar('speckleWindowSize','disp',3,25,nothing)
cv2.createTrackbar('disp12MaxDiff','disp',5,25,nothing)
cv2.createTrackbar('minDisparity','disp',5,25,nothing)

# Creating an object of StereoBM algorithm
stereo = cv2.StereoBM_create()

while True:

	# Capturing and storing left and right camera images
	retL, imgL= CamL.read()
	retR, imgR= CamR.read()
	
	# Proceed only if the frames have been captured
	if retL and retR:
		imgR_gray = cv2.cvtColor(imgR,cv2.COLOR_BGR2GRAY)
		imgL_gray = cv2.cvtColor(imgL,cv2.COLOR_BGR2GRAY)

		# Applying stereo image rectification on the left image
		Left_nice= cv2.remap(imgL_gray,
							Left_Stereo_Map_x,
							Left_Stereo_Map_y,
							cv2.INTER_LANCZOS4,
							cv2.BORDER_CONSTANT,
							0)
		
		# Applying stereo image rectification on the right image
		Right_nice= cv2.remap(imgR_gray,
							Right_Stereo_Map_x,
							Right_Stereo_Map_y,
							cv2.INTER_LANCZOS4,
							cv2.BORDER_CONSTANT,
							0)

		# Updating the parameters based on the trackbar positions
		numDisparities = cv2.getTrackbarPos('numDisparities','disp')*16
		blockSize = cv2.getTrackbarPos('blockSize','disp')*2 + 5
		preFilterType = cv2.getTrackbarPos('preFilterType','disp')
		preFilterSize = cv2.getTrackbarPos('preFilterSize','disp')*2 + 5
		preFilterCap = cv2.getTrackbarPos('preFilterCap','disp')
		textureThreshold = cv2.getTrackbarPos('textureThreshold','disp')
		uniquenessRatio = cv2.getTrackbarPos('uniquenessRatio','disp')
		speckleRange = cv2.getTrackbarPos('speckleRange','disp')
		speckleWindowSize = cv2.getTrackbarPos('speckleWindowSize','disp')*2
		disp12MaxDiff = cv2.getTrackbarPos('disp12MaxDiff','disp')
		minDisparity = cv2.getTrackbarPos('minDisparity','disp')
		
		# Setting the updated parameters before computing disparity map
		stereo.setNumDisparities(numDisparities)
		stereo.setBlockSize(blockSize)
		stereo.setPreFilterType(preFilterType)
		stereo.setPreFilterSize(preFilterSize)
		stereo.setPreFilterCap(preFilterCap)
		stereo.setTextureThreshold(textureThreshold)
		stereo.setUniquenessRatio(uniquenessRatio)
		stereo.setSpeckleRange(speckleRange)
		stereo.setSpeckleWindowSize(speckleWindowSize)
		stereo.setDisp12MaxDiff(disp12MaxDiff)
		stereo.setMinDisparity(minDisparity)

		# Calculating disparity using the StereoBM algorithm
		disparity = stereo.compute(Left_nice,Right_nice)
		# NOTE: Code returns a 16bit signed single channel image,
		# CV_16S containing a disparity map scaled by 16. Hence it 
		# is essential to convert it to CV_32F and scale it down 16 times.

		# Converting to float32 
		disparity = disparity.astype(np.float32)

		# Scaling down the disparity values and normalizing them 
		disparity = (disparity/16.0 - minDisparity)/numDisparities

		# Displaying the disparity map
		cv2.imshow("disp",disparity)

		# Close window using esc key
		if cv2.waitKey(1) == 27:
			break
	
	else:
		CamL= cv2.VideoCapture(CamL_id)
		CamR= cv2.VideoCapture(CamR_id)

https://learnopencv.com/depth-perception-using-stereo-camera-python-c/

**Number of disparities (numDisparities)**: Sets the range of disparity values to be searched. The overall range is from minimum disparity value to minimum disparity value + number of disparities. The following pair of images shows the disparity map calculated for two different disparity ranges. It is clearly visible that increasing the number of disparities increases the accuracy of the disparity map.

**Block size (blockSize)**: Size of the sliding window used for block matching to find corresponding pixels in a rectified stereo image pair. A higher value indicates a larger window size. The following GIF indicates that increasing this parameter results in more smooth disparity maps.

**Pre-Filter Type (preFilterType)**: Parameter to decide the type of pre-filtering to be applied to the images before passing to the block matching algorithm. This step enhances the texture information and improves the results of the block matching algorithm. The filter type can be CV_STEREO_BM_XSOBEL or CV_STEREO_BM_NORMALIZED_RESPONSE.


**Pre-filter size (preFilterSize)**: Window size of the filter used in the pre-filtering stage.


**Pre-filter cap (preFilterCap)**: Limits the filtered output to a specific value.


**Minimum disparity (minDisparity)**: The minimum value of the disparity to be searched. In most scenarios it is set to zero. It can also be set to negative value depending on the stereo camera setup.


**Texture threshold (textureThreshold)**: Filters out areas that do not have enough texture information for reliable matching.


**Uniqueness Ratio (uniquenessRatio)**: Another post-filtering step. If the best matching disparity is not sufficiently better than every other disparity in the search range, the pixel is filtered out. The following GIF depicts that increasing the uniqueness ratio increases the number of pixels that are filtered out.

**Speckle range (speckleRange) and speckle window size (speckleWindowSize)**: Speckles are produced near the boundaries of the objects, where the matching window catches the foreground on one side and the background on the other. To get rid of these artifacts we apply speckle filter which has two parameters. The speckle range defines how close the disparity values should be to be considered as part of the same blob. The speckle window size is the number of pixels below which a disparity blob is dismissed as “speckle”.

**vdisp12MaxDiff**: Pixels are matched both ways, from the left image to the right image and from the right image to left image. disp12MaxDiff defines the maximum allowable difference between the original left pixel and the back-matched pixel.


OpenCv also provides StereoSGBM Is a Semi-Global Block Matching

In [ ]:
#Figure parameters
width = 10
height = 10
fig = plt.figure(figsize=(10, 8))
columns = 2
rows = 1

#EXAMPLE OF DEPTH ESTIMATION
left = cv2.imread('./../../Data/Stereo_conveyor_without_occlusions_undistorted/left/1585434283_489372015_Left.png')
imgleft=cv2.imread('./../../Data/Stereo_conveyor_without_occlusions_undistorted/left/1585434283_489372015_Left.png',0)
left = cv2.cvtColor(left, cv2.COLOR_BGR2GRAY)

fig.add_subplot(1, 2, 1)
plt.imshow(left)
plt.title("Left")

right = cv2.imread('./../../Data/Stereo_conveyor_without_occlusions_undistorted/right/1585434283_489372015_Right.png')
imgright=cv2.imread('./../../Data/Stereo_conveyor_without_occlusions_undistorted/right/1585434283_489372015_Right.png',0)
right = cv2.cvtColor(right, cv2.COLOR_BGR2GRAY)

fig.add_subplot(1, 2, 2)
plt.imshow(right)
plt.title("Right")
plt.show()

depth = Depth(left, right, positionOnLeft=(450,500), halfWindow=50)

print("Depth: ", depth, "mm")

In [ ]:
min_disp = 1*16 # 7
num_disp = 4*16 # 3*16
block_size = 3*2+5 # NEEDS TO BE ODD. LARGER= SMOOTHER AND LESS ACCURATE
preFilterType = 1
stereo = cv2.StereoBM_create(numDisparities = num_disp, blockSize = block_size)
#stereo.setPreFilterType(cv2.CV_STEREO_BM_XSOBEL)
stereo.setPreFilterSize(10*2+5)
stereo.setPreFilterCap(40)
stereo.setTextureThreshold(50)
stereo.setUniquenessRatio(40) # 3
stereo.setSpeckleRange(3*2) # 3
stereo.setMinDisparity(min_disp)
stereo.setDisp12MaxDiff(20) # 200
stereo.setSpeckleWindowSize(100) # 3


"""
stereo.setNumDisparities(numDisparities)
stereo.setBlockSize(blockSize)
stereo.setPreFilterType(preFilterType)
stereo.setPreFilterSize(preFilterSize)
stereo.setPreFilterCap(preFilterCap)
stereo.setTextureThreshold(textureThreshold)
stereo.setUniquenessRatio(uniquenessRatio)
stereo.setSpeckleRange(speckleRange)
stereo.setSpeckleWindowSize(speckleWindowSize)
stereo.setDisp12MaxDiff(disp12MaxDiff)
stereo.setMinDisparity(minDisparity)

"""
disparity = stereo.compute(imgleft, imgright)

min = disparity.min()
max = disparity.max()
disparity = np.uint8(6400 * (disparity - min) / (max - min))

cv2.imshow('disparittet', np.hstack((imgleft, imgright, disparity)))
plt.imshow(disparity)